# Using LogReg for feature selection

For this quickstart, you will need to install lamindb in addition to modlyn. Also, init a lamindb instance to query a test dataset.

```
pip install modlyn lamindb
lamin init --storage test-modlyn
```

In [ ]:
import modlyn as mn
import lamindb as ln

ln.track()

In [ ]:
artifact = ln.Artifact.using("laminlabs/arrayloader-benchmarks").get("D21D2K8697CY8tHE0001")
adata = artifact.load()
adata

In [ ]:
adata.obs.value_counts("cell_line")

In [ ]:
logreg = mn.models.SimpleLogReg(
    adata=adata,
    label_column="cell_line",    
    learning_rate=1e-1,
)
logreg.fit(
    adata_train=adata,
    adata_val=adata[:100],
    train_dataloader_kwargs={
        "batch_size": 8,
    },
    max_epochs=4,
)

In [ ]:
logreg.plot_losses()

In [ ]:
logreg.plot_classification_report(adata)

In [ ]:
# ln.finish()